In [1]:
import pandas as pd
import os


In [2]:
# script consts
PHEN_START = 2
VALUE_AT_END_OF_PHENOTYPES = 'const'
PLINK_PATH = "/cs/usr/nadavb/third_party/plink2"
BATCH_LENGTH = 250
MAX_JOBS_BEFORE_SUBMITTING_NEW_ARRAY = 200


In [3]:
# # input files
# PHEN_PATH = "/sci/nosnap/michall/roeizucker/Hypothyroidism_hackaton/ukbb_dataset_filtered.csv"
# GWAS_PATH = "/sci/nosnap/michall/roeizucker/Hypothyroidism_hackaton/GWAS_FOR_PRS"
# PHENOTYPES = ["E03_F_reduced","E03_M_reduced","E03_reduced",
#              "E04_F_reduced","E04_M_reduced","E04_reduced",
#              "E05_F_reduced","E05_M_reduced","E05_reduced",
#              "E06_F_reduced","E06_M_reduced","E06_reduced"
#              "E07_F_reduced","E07_M_reduced","E07_reduced"]
# RESULT_SUFFIX = "PHENO1.glm.logistic"
# COVARS = []
# SPLIT_TO_TRAIN_TEST = True
# CREATE_PHENOTYPE_FILES = False
# CREATE_COVAR_FILE = True
# TRAIN_RATIO = 0.8
# input files
PHEN_PATH = "/cs/labs/michall/roeizucker/IIH/PWAS/ukbb_dataset_updated.csv"
GWAS_PATH = "/cs/labs/michall/roeizucker/IIH/GWAS4"
PHENOTYPES = ["updated_Both","updated_PAP","updated_IIH"]
RESULT_SUFFIX = "PHENO1.glm.logistic"
COVARS = []
SPLIT_TO_TRAIN_TEST = False
CREATE_PHENOTYPE_FILES = True
CREATE_COVAR_FILE = True
TRAIN_RATIO = 0.8

In [4]:
!mkdir {GWAS_PATH}

In [5]:
!head /cs/labs/michall/roeizucker/SKAT_experiments/SetID/ukbb_dataset_withIIH.csv

eid2,sample_index,eid,F329,F329_F,F329_M,H26,H26_F,H26_M,H269,H269_F,H269_M,I10,I10_F,I10_M,const,sex,year_of_birth,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40,AC_stockport_pilot,AC_manchester,AC_oxford,AC_cardiff,AC_glasgow,AC_edinburgh,AC_stoke,AC_reading,AC_bury,AC_newcastle,AC_bristol,AC_barts,AC_nottingham,AC_sheffield,AC_liverpool,AC_middlesborough,AC_hounslow,AC_croydon,AC_birmingham,AC_swansea,AC_wrexham,AC_cheadle_revisit,AC_cheadle_imaging,AC_reading_imaging,AC_newcastle_imaging,batch_Batch_b001,batch_Batch_b002,batch_Batch_b003,batch_Batch_b004,batch_Batch_b005,batch_Batch_b006,batch_Batch_b007,batch_Batch_b008,batch_Batch_b009,batch_Batch_b010,batch_Batch_b011,batch_Batch_b012,batch_Batch_b013,batch_Batch_b014,batch_Batch_b015,batch_Batch_b016,batch_Batch_b017,batch_Batch_b018,batch_Batch_b019,batch_Batch_b020,batch_Batch_b021,ba

In [6]:
def create_files(gwas_path,phen_path,phenotypes):
    !mkdir {gwas_path}
    !mkdir {gwas_path}/results
    !mkdir {gwas_path}/phenotypes
    !cp /cs/labs/michall/roeizucker/10krun/runs/0:11/GWAS_delete_me/covariates.txt {gwas_path}/covariates.txt
    !cd {gwas_path}
    dataset = pd.read_csv(phen_path)
    if not CREATE_PHENOTYPE_FILES:
        return
    for phenotype_col in phenotypes:
        print(phenotype_col)
        file_name = phenotype_col.lower().replace(' ', '_').replace('-', '_') + '.txt'
        # use eid as fid, because we take one representitive of each family
        values = dataset[['eid', 'eid', phenotype_col]].dropna()
        if SPLIT_TO_TRAIN_TEST:
            values_case = values[values[phenotype_col] == 1]
            values_control = values[values[phenotype_col]== 0] 
            values_case = values_case.sample(int(len(values_case) * TRAIN_RATIO))
            values_control = values_control.sample(int(len(values_control) * TRAIN_RATIO))
            values = pd.concat([values_case,values_control])
        values.to_csv(os.path.join(f"{gwas_path}/phenotypes", file_name), \
            header = False, index = False, sep = '\t')

        # An array is created with all the values needed for running GWAS.
# The array will be sorted in batches for better file accessing
def create_the_data_file(phenotype_cols,gwas_path,data_file_path):
    values = []
    last_counter = 0
    cur_phen_counter = 0
    counter = 0
    for phen in phenotype_cols:
        base_path =  gwas_path
        for j in range(1,23):
            if os.path.exists(os.path.join(base_path ,f"results/{phen.lower()}_chr{j}.PHENO1.glm.linear")) or \
            os.path.exists(os.path.join(base_path ,f"results/{phen.lower()}_chr{j}.PHENO1.glm.logistic")):
#                 print("a")
                continue
            
            # TODO: change location of plink files to be const
            values.append([counter,os.path.join(base_path, "phenotypes/"+ phen.lower() + ".txt"),
                           os.path.join(base_path, f"results/{phen.lower()}_chr{j}"),
                           f"/cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}"
                           ,os.path.join(base_path ,"covariates.txt"),j])
            counter+=1
        cur_phen_counter+=1
        if cur_phen_counter%BATCH_LENGTH == 0:
            values[last_counter:counter] = sorted(values[last_counter:counter],key = lambda x:x[-1])
            last_counter = counter
    values[last_counter:counter] = sorted(values[last_counter:counter],key = lambda x:x[-1])
    df = pd.DataFrame(values, columns=["unsorted_counter","phenotype_path","output_path","partial_chromosome_file_path", "covariates","chr"])
    df.to_csv(data_file_path)


def write_mediator_script(mediator_script_path,data_file_path,gwas_path):
    with open(mediator_script_path, "w") as mediator_file:
        mediator_file.write(f'''import pandas as pd
import sys
import os
import os.path
butch_num = int(sys.argv[1]) 
curr_task = int(sys.argv[2])
BATCH_LENGTH = {BATCH_LENGTH}
SKIP_POINT = {MAX_JOBS_BEFORE_SUBMITTING_NEW_ARRAY}
PLINK_PATH = "/cs/usr/nadavb/third_party/plink2"

data_file_path = "{data_file_path}"
df = pd.read_csv(data_file_path)
print(df.loc[butch_num * BATCH_LENGTH + curr_task])
val = df.loc[butch_num * BATCH_LENGTH + curr_task]
location = butch_num * BATCH_LENGTH + curr_task
if curr_task == SKIP_POINT and location < 30500:
    new_batch  = butch_num + 1
    if not os.path.isfile("{gwas_path}_" + str(new_batch) + "_flag"):
        os.system("sbatch --array=0-"+str(BATCH_LENGTH - 1)+" --mem=12g -c10 --time=3-0 --killable --requeue --wrap=\\"{gwas_path}master.sh "+str(new_batch)+"\\"")
if location < 30500:
    print(PLINK_PATH + " --bed " + df.loc[location].partial_chromosome_file_path + ".bed --bim " + df.loc[location].partial_chromosome_file_path + ".bim --fam "+df.loc[location].partial_chromosome_file_path + ".fam --pheno " + df.loc[location].phenotype_path + " --covar " + df.loc[location].covariates + "  --out  " + df.loc[location].output_path + " --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000")
    os.system(PLINK_PATH + " --bed " + df.loc[location].partial_chromosome_file_path + ".bed --bim " + df.loc[location].partial_chromosome_file_path + ".bim --fam "+df.loc[location].partial_chromosome_file_path + ".fam --pheno " + df.loc[location].phenotype_path + " --covar " + df.loc[location].covariates + "  --out  " + df.loc[location].output_path + " --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000")
    print("done!")
''')

def write_master_script(master_script_path,mediator_script_path,gwas_path):
    with open(master_script_path, "w") as master_file:
        master_file.write(f'''
FILE={gwas_path}_$1_flag
if ! test "$FILE" 
then 
    touch {gwas_path}_$1_flag
fi
python {mediator_script_path} $1 $SLURM_ARRAY_TASK_ID
''')
    !chmod 744 {master_script_path}


In [7]:
# data_file_path = os.path.join(GWAS_PATH, "data_file.csv")
# create_the_data_file(PHENOTYPES,GWAS_PATH,data_file_path)

In [8]:
# master_script_path = os.path.join(GWAS_PATH , "master.sh")
# print(f'''how to run:
# sbatch --array=0-263 --mem=12g -c10 --time=3-0 --requeue --killable --wrap="{master_script_path} 0"''')

In [9]:
# script body (will be transferred to another file)


# TODO: add verbous argument, and better explain the output

# set input 
# TODO: change so it is accepted as params
phen_path = PHEN_PATH
gwas_path = GWAS_PATH

# do stuff
phenotype_cols = list(pd.read_csv(PHEN_PATH))
phenotype_cols = phenotype_cols[2:phenotype_cols.index(VALUE_AT_END_OF_PHENOTYPES)]

# we will use the phenotypes parameter only if it exists
if len (PHENOTYPES) > 0:
    phenotype_cols = PHENOTYPES
    
# TODO: chnage so names are consts
data_file_path = os.path.join(gwas_path, "data_file.csv")
mediator_script_path = os.path.join(gwas_path , "mediator.py")
master_script_path = os.path.join(gwas_path , "master.sh")

create_files(gwas_path,phen_path,phenotype_cols)
create_the_data_file(phenotype_cols,gwas_path,data_file_path)
write_mediator_script(mediator_script_path,data_file_path,gwas_path)
write_master_script(master_script_path,mediator_script_path,gwas_path)

# TODO: add output folder
print(f'''how to run:
sbatch --array=0-263 --mem=12g -c10 --time=3-0 --requeue --killable --wrap="{master_script_path} 0"''')

mkdir: cannot create directory ‘/cs/labs/michall/roeizucker/IIH/GWAS4’: File exists
updated_Both
updated_PAP
updated_IIH
how to run:
sbatch --array=0-263 --mem=12g -c10 --time=3-0 --requeue --killable --wrap="/cs/labs/michall/roeizucker/IIH/GWAS4/master.sh 0"


In [10]:
# sbatch --array=0-33 --mem=12g -c10 --time=3-0 --requeue --killable --wrap="/cs/labs/michall/roeizucker/IIH/GWAS2/master.sh 0"
sbatch --array=0-60 --mem=12g -c10 --time=3-0 --requeue --killable --wrap="/cs/labs/michall/roeizucker/IIH/GWAS3/master.sh 0"


SyntaxError: invalid syntax (2859877951.py, line 2)

In [ ]:
# master_script_path = priority_gwas_dir + "master.sh"

# (mediator_script_path,data_file_path,gwas_path)
master_script_path

In [ ]:
pd.read_csv(data_file_path)

In [ ]:
print(f'''how to run:
sbatch --array=0-263 --mem=12g -c10 --time=3-0 --requeue --killable --wrap="{master_script_path} 0"''')

## combines GWAS results

In [ ]:
import glob, os
# path = "/cs/labs/michall/roeizucker/10krun/runs/48:59/GWAS/results"
# os.chdir(path)
path = os.path.join(GWAS_PATH,"results")
# files = []
# for file in glob.glob("*.glm.*"):
#     files.append(file)
phens = PHENOTYPES
for phen in phens:
    dataframes = []
    for j in range(1,23):
#         change so logistic/liner is detrmined generically
        file_name = os.path.join(path,f"{phen.lower()}_chr{j}.{RESULT_SUFFIX}")
        try:
            df = pd.read_csv(file_name,sep="\t")
            dataframes.append(df)
        except pd.errors.EmptyDataError:
            print(file_name)
            continue
        except FileNotFoundError:
            print(file_name)
            continue
    if len(dataframes) == 0:
        continue
    master_df = pd.concat(dataframes)
    master_df.to_csv(f"{os.path.join(path,phen)}.csv",index=False)
    print(f"{os.path.join(path,phen)}.csv")

In [ ]:
os.path.join(path,phen)